<a href="https://colab.research.google.com/github/Saranyanv/test/blob/master/test_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [0]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
X_train=X_train[0:600]
y_train=y_train[0:600]
X_test=X_test[0:100]
y_test=y_test[0:100]

In [0]:
import numpy as np
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[0:50], X_train[50:500]
y_valid, y_train = y_train[0:50], y_train[50:500]

In [0]:
n_inputs = 28*28  # MNIST
n_hidden1_1 = 300
n_hidden1_2 = 100
n_outputs = 10

In [0]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [0]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1_1, name="hidden1",
                              activation=tf.nn.softmax)
    hidden2 = tf.layers.dense(hidden1, n_hidden1_2, name="hidden2",
                              activation=tf.nn.softmax)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [13]:
print ("Acc=",accuracy)

Acc= Tensor("eval/Mean:0", shape=(), dtype=float32)


In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [23]:
n_epochs = 20
n_batches = 50

graph1 = tf.Graph()
with graph1.as_default():
  session1 = tf.Session()
  with session1.as_default():
      init.run()
      for epoch in range(n_epochs):
          for X_batch, y_batch in shuffle_batch(X_train, y_train, n_batches):
              sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
          acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
          acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
          print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

ValueError: ignored

In [28]:
#show_graph(tf.get_default_graph())
show_graph(tf.get_default_graph().as_graph_def())